In [1]:
# Install with `pip install impectPy numpy pandas`
import impectPy as ip
import numpy as np
import pandas as pd

In [ ]:
# Set login credentials
username = "yourUsername"
password = "yourPassword"

# Fetch access token
token = ip.getAccessToken(username=username, password=password)

In [ ]:
# Set iterationId
iterationId = 1385

# Fetch matches for iteration
matches = ip.getMatches(iterationId, token=token)
matches.head()

In [ ]:
# Fetch prediction model coefficients
coefficients = ip.getSquadCoefficients(iterationId, token=token)
coefficients.head()

In [ ]:
# Prepare columns for merging
matches["date"] = pd.to_datetime(matches["scheduledDate"]).dt.tz_localize(None).dt.normalize()
matches["homeSquadId"] = matches["homeSquadId"].astype("int64")
matches["awaySquadId"] = matches["awaySquadId"].astype("int64")
coefficients["date"] = pd.to_datetime(coefficients["date"]).dt.normalize()
coefficients["squadId"] = coefficients["squadId"].astype("int64")

# Sort by date
matches = matches.sort_values("date")
coefficients = coefficients.sort_values("date")

# Merge competition-specific coefficients using the most recent date
matches = pd.merge_asof(
    matches,
    coefficients[["date", "interceptCoefficient", "homeCoefficient", "competitionCoefficient"]].drop_duplicates("date"),
    on="date",
    direction="backward"
)

# Merge squad-specific coefficients using the most recent date
def get_squad_coeffs(row, coeff_df, squad_id_col) -> pd.Series:
    squad_id = row[squad_id_col]
    match_date = row["date"]
    squad_coeffs = coeff_df[(coeff_df["squadId"] == squad_id) & (coeff_df["date"] <= match_date)]
    if len(squad_coeffs) > 0:
        latest = squad_coeffs.sort_values("date").iloc[-1]
        return pd.Series({"attack": latest["attackCoefficient"], "defense": latest["defenseCoefficient"]})
    return pd.Series({"attack": None, "defense": None})

# Merge homeSquad coefficients
home_coeffs = matches.apply(lambda row: get_squad_coeffs(row, coefficients, "homeSquadId"), axis=1)
matches["attackCoefficientHome"] = home_coeffs["attack"]
matches["defenseCoefficientHome"] = home_coeffs["defense"]

# Merge awaySquad coefficients
away_coeffs = matches.apply(lambda row: get_squad_coeffs(row, coefficients, "awaySquadId"), axis=1)
matches["attackCoefficientAway"] = away_coeffs["attack"]
matches["defenseCoefficientAway"] = away_coeffs["defense"]

matches.head()

In [ ]:
# Compute predictions
matches["predHome"] = np.exp(
    matches["interceptCoefficient"] +
    matches["homeCoefficient"] +
    matches["competitionCoefficient"] +
    matches["attackCoefficientHome"] +
    matches["defenseCoefficientAway"]
)
matches["predAway"] = np.exp(
    matches["interceptCoefficient"] +
    matches["competitionCoefficient"] +
    matches["attackCoefficientAway"] +
    matches["defenseCoefficientHome"]
)

matches.head()